# 15 - HDFS Fundamentals

Hadoop Distributed File System - rozproszony system plików stanowiący fundament ekosystemu Hadoop.

**Tematy:**
- Architektura HDFS: NameNode, DataNode, bloki
- HDFS CLI - podstawowe operacje
- Upload danych MovieLens do HDFS
- Spark + HDFS - odczyt i zapis
- Replication factor i fault tolerance
- HDFS Web UI i monitoring
- Porównanie: local fs vs HDFS vs PostgreSQL JDBC

## 1. Architektura HDFS

```
                  ┌──────────────┐
                  │   NameNode   │  ← metadane (nazwy plików, bloki, lokalizacje)
                  │   (master)   │  ← Single Point of Failure (dlatego HA!)
                  └──────┬───────┘
                         │
            ┌────────────┼────────────┐
            ▼            ▼            ▼
     ┌────────────┐┌────────────┐┌────────────┐
     │ DataNode 1 ││ DataNode 2 ││ DataNode 3 │
     │            ││            ││            │
     │ Block A    ││ Block A    ││ Block B    │  ← dane (bloki 128MB)
     │ Block B    ││ Block C    ││ Block C    │  ← replication factor=2
     └────────────┘└────────────┘└────────────┘
```

### Kluczowe koncepty:
- **Block size**: domyślnie 128MB (duże bloki → mniej metadanych w NameNode)
- **Replication factor**: domyślnie 3 (każdy blok na 3 DataNodeach)
- **Write-once, read-many**: pliki nie są edytowane, tylko dopisywane lub nadpisywane
- **Data locality**: Spark przetwarza dane tam gdzie są (unika transferu sieciowego)

## 2. Setup i połączenie

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("15_HDFS_Fundamentals") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.1") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "7g") \
    .config("spark.driver.host", "recommender-jupyter") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .getOrCreate()

# HDFS connection
HDFS_URL = "hdfs://namenode:9000"
HDFS_DATA = f"{HDFS_URL}/data/movielens"

# PostgreSQL
jdbc_url = "jdbc:postgresql://postgres:5432/recommender"
jdbc_props = {"user": "recommender", "password": "recommender", "driver": "org.postgresql.Driver"}

## 3. HDFS CLI

HDFS ma CLI bardzo podobne do standardowych poleceń Unix.

In [ ]:
# Podstawowe komendy HDFS (uruchamiane z kontenera z hadoop)
# Tutaj symulujemy via pyspark

# Sprawdź konfigurację Hadoop z poziomu Spark
hadoop_conf = spark.sparkContext._jsc.hadoopConfiguration()
print(f"Default FS: {hadoop_conf.get('fs.defaultFS', 'not set')}")
print(f"Replication: {hadoop_conf.get('dfs.replication', 'not set')}")
print(f"Block size: {hadoop_conf.get('dfs.blocksize', 'not set')}")

In [ ]:
%%bash
# HDFS CLI - podstawowe operacje
# (wymaga dostępu do komendy hdfs w kontenerze)

# Listowanie katalogu root
hdfs dfs -ls /

# Tworzenie katalogu
hdfs dfs -mkdir -p /data/movielens/raw
hdfs dfs -mkdir -p /data/movielens/bronze
hdfs dfs -mkdir -p /data/movielens/silver
hdfs dfs -mkdir -p /data/movielens/gold

# Sprawdź strukturę
hdfs dfs -ls -R /data/movielens/

In [ ]:
%%bash
# Upload pliku do HDFS
# hdfs dfs -put /local/path/rating.csv /data/movielens/raw/

# Download z HDFS
# hdfs dfs -get /data/movielens/raw/rating.csv /local/path/

# Podgląd pliku (pierwsze linie)
# hdfs dfs -head /data/movielens/raw/rating.csv

# Rozmiar pliku
# hdfs dfs -du -h /data/movielens/raw/

# Informacje o blokach pliku
# hdfs fsck /data/movielens/raw/rating.csv -blocks -locations

# Usuwanie
# hdfs dfs -rm -r /data/movielens/raw/rating.csv

# Zmiana replication factor
# hdfs dfs -setrep 2 /data/movielens/raw/rating.csv

echo "HDFS CLI commands reference (uncomment to run)"

## 4. Upload danych do HDFS przez Spark

Zamiast CLI, możemy użyć Spark do zapisu danych na HDFS.

In [ ]:
# Załaduj dane z PostgreSQL
ratings = spark.read.jdbc(
    jdbc_url, "movielens.ratings", properties=jdbc_props,
    column="user_id", lowerBound=1, upperBound=300000, numPartitions=10
)
movies = spark.read.jdbc(jdbc_url, "movielens.movies", properties=jdbc_props)

# Zapisz na HDFS jako Parquet
ratings.write.mode("overwrite").parquet(f"{HDFS_DATA}/raw/ratings")
movies.write.mode("overwrite").parquet(f"{HDFS_DATA}/raw/movies")

print(f"Ratings zapisane na HDFS: {ratings.count()} rows")
print(f"Movies zapisane na HDFS: {movies.count()} rows")

In [ ]:
# Odczytaj z HDFS
ratings_hdfs = spark.read.parquet(f"{HDFS_DATA}/raw/ratings")
movies_hdfs = spark.read.parquet(f"{HDFS_DATA}/raw/movies")

print(f"Ratings z HDFS: {ratings_hdfs.count()} rows")
ratings_hdfs.show(5)

In [ ]:
# Informacje o plikach na HDFS (via Hadoop FileSystem API)
fs = spark.sparkContext._jvm.org.apache.hadoop.fs.FileSystem.get(
    spark.sparkContext._jvm.java.net.URI(HDFS_URL),
    spark.sparkContext._jsc.hadoopConfiguration()
)

path = spark.sparkContext._jvm.org.apache.hadoop.fs.Path(f"{HDFS_DATA}/raw/ratings")
status = fs.listStatus(path)

print(f"Pliki w {HDFS_DATA}/raw/ratings/:")
total_size = 0
for s in status:
    size_mb = s.getLen() / 1024 / 1024
    total_size += s.getLen()
    print(f"  {s.getPath().getName():<40} {size_mb:.1f} MB  replication={s.getReplication()}")

print(f"\nTotal: {total_size / 1024 / 1024:.1f} MB")

## 5. Benchmark: Local vs HDFS vs PostgreSQL

In [ ]:
import time

# Zapisz też lokalnie dla porównania
ratings.write.mode("overwrite").parquet("/tmp/ratings_local")

def benchmark(name, func, runs=3):
    times = []
    for _ in range(runs):
        start = time.time()
        func()
        times.append(time.time() - start)
    avg = sum(times) / len(times)
    print(f"{name:<35} {avg:.2f}s (avg of {runs})")
    return avg

query = lambda df: df.filter(col("rating") >= 4.0).groupBy("movie_id").count().count()

print("=== Read + filter + groupBy benchmark ===")
benchmark("Local Parquet (/tmp)",
    lambda: query(spark.read.parquet("/tmp/ratings_local")))

benchmark("HDFS Parquet",
    lambda: query(spark.read.parquet(f"{HDFS_DATA}/raw/ratings")))

benchmark("PostgreSQL JDBC",
    lambda: query(spark.read.jdbc(jdbc_url, "movielens.ratings", properties=jdbc_props)))

## 6. Replication i Fault Tolerance

In [ ]:
# Zapisz z różnym replication factor
for rep in [1, 2, 3]:
    spark.sparkContext._jsc.hadoopConfiguration().set("dfs.replication", str(rep))
    ratings.limit(10000).write.mode("overwrite") \
        .parquet(f"{HDFS_DATA}/test_replication/rep_{rep}")

# Sprawdź rozmiar na dysku per replication
for rep in [1, 2, 3]:
    path = spark.sparkContext._jvm.org.apache.hadoop.fs.Path(
        f"{HDFS_DATA}/test_replication/rep_{rep}")
    content_summary = fs.getContentSummary(path)
    logical = content_summary.getLength() / 1024 / 1024
    physical = content_summary.getSpaceConsumed() / 1024 / 1024
    print(f"Replication={rep}: logical={logical:.1f}MB, physical={physical:.1f}MB (ratio={physical/logical:.1f}x)")

# Przywróć domyślny replication
spark.sparkContext._jsc.hadoopConfiguration().set("dfs.replication", "2")

### Co się dzieje gdy DataNode padnie?

1. NameNode wykrywa brak heartbeatu (timeout 10 min)
2. Bloki z tego DataNode są oznaczone jako under-replicated
3. NameNode zleca replikację na inny DataNode
4. Dane są nadal dostępne z pozostałych replik

**Replication factor = 3 → klaster przeżywa utratę 2 DataNodeów jednocześnie.**

## 7. HDFS Web UI

NameNode Web UI: **http://namenode:9870**

Co można zobaczyć:
- Overview: pojemność, wykorzystanie, live/dead DataNodes
- Datanodes: status każdego DataNode
- Browse: przeglądarka plików HDFS
- Block Scanner: status replikacji bloków

### Zadanie 1
1. Otwórz HDFS Web UI
2. Znajdź plik ratings w `/data/movielens/raw/`
3. Sprawdź na których DataNodeach są bloki tego pliku
4. Ile bloków ma plik? Jaki jest ich rozmiar?

In [ ]:
# Twoje rozwiązanie / notatki:


## 8. Data Locality w Spark

Spark próbuje uruchomić task na tym samym node gdzie są dane (data locality levels):

| Level | Opis | Szybkość |
|-------|------|---------|
| PROCESS_LOCAL | Dane w pamięci tego samego executor | Najszybsze |
| NODE_LOCAL | Dane na tym samym node (HDFS DataNode) | Szybkie |
| RACK_LOCAL | Dane w tym samym racku | OK |
| ANY | Dane na dowolnym node | Najwolniejsze (transfer sieciowy) |

In [ ]:
# Sprawdź locality w Spark UI → Stages → Task Locality Level
# Po uruchomieniu tego joba, otwórz Spark UI i sprawdź locality

result = spark.read.parquet(f"{HDFS_DATA}/raw/ratings") \
    .groupBy("movie_id") \
    .agg(avg("rating"), count("*")) \
    .count()

print(f"Result: {result}")
print(f"Sprawdź Spark UI: {spark.sparkContext.uiWebUrl}")
print("→ Stages → kliknij stage → Locality Level")

## Zadanie końcowe

Zbuduj Medallion Architecture na HDFS:

1. **Bronze**: załaduj surowe dane z PostgreSQL → HDFS `/data/movielens/bronze/` (Parquet)
2. **Silver**: oczyść dane (deduplikacja, nulls, enrichment) → HDFS `/data/movielens/silver/`
3. **Gold**: agregaty (movie_stats, user_profiles) → HDFS `/data/movielens/gold/`
4. Porównaj czas odczytu Gold z HDFS vs Gold z PostgreSQL
5. Sprawdź rozmiar każdej warstwy na HDFS (`du -h`)

In [ ]:
# Twoje rozwiązanie:


In [ ]:
spark.stop()